#### Installing necessary libs

In [12]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


#### Importing OpenAI gym and other dependencies and testing environments

In [13]:
import random
import gym
import os
os.environ['SDL_VIDEODRIVER']='dummy'

#### Declaring environment and creating states

In [4]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
print("number os states:", states)
actions = env.action_space.n
print("number os actions:", actions)

NameError: name 'gym' is not defined

#### Creating Iteration with n episodes and passing the action

In [15]:
episodes = 3
for episode in range(1, episodes+10):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))

Episode:1 Score:61.0
Episode:2 Score:22.0
Episode:3 Score:16.0
Episode:4 Score:26.0
Episode:5 Score:16.0
Episode:6 Score:15.0
Episode:7 Score:23.0
Episode:8 Score:18.0
Episode:9 Score:16.0
Episode:10 Score:22.0
Episode:11 Score:19.0
Episode:12 Score:10.0


#### Modelling with Keras

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [2]:
import tensorflow

def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(units=24, activation='relu' ))
    model.add(Dense(units=24, activation='relu'))
    model.add(Dense(units=actions, activation='linear'))
    return model

In [3]:
model = build_model(states, actions)
model.summary()

NameError: name 'states' is not defined

#### Initialising Agent with Keras - Reinforced Learning

In [4]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [7]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [8]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)

NameError: name 'model' is not defined

In [ ]:
scores = dqn.test(env, nb_episodes=15, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

##### references
##### 1. https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic
##### 2. https://gsurma.medium.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288
##### 3. https://github.com/ChaithanyaVamshi/CartPole-DQN-Reinforcementlearning
##### 4. https://github.com/alishafique3/CartPole-DQN-model-in-Google-Colab/blob/main/RL_Classical_Framework.ipynb
##### 5. https://stackoverflow.com/questions/tagged/dqn